In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import glob as glob
import cv2
import os
import shutil
from sklearn.model_selection import train_test_split

In [2]:
train_labels = pd.read_csv("train_labels.csv")
train_labels

,id,label
0,0,1
1,1,13
2,2,13
3,3,14
4,4,6
...,...,...
15995,15995,2
15996,15996,15
15997,15997,3
15998,15998,9


In [4]:
df_train, df_val = train_test_split(train_labels, test_size=0.10, random_state=101, stratify=train_labels["label"])

In [61]:
df_train["label"].value_counts()

8     953
6     940
0     929
12    918
5     911
11    910
13    906
9     900
3     894
14    893
15    889
10    877
1     872
2     871
4     869
7     868
Name: label, dtype: int64

In [62]:
df_val["label"].value_counts()

8     106
6     104
0     103
12    102
13    101
11    101
5     101
9     100
14     99
3      99
15     99
7      97
2      97
10     97
4      97
1      97
Name: label, dtype: int64

# Run these below cells only when you do not have train_dir and val_dir created

In [56]:
os.mkdir("train_dir")
os.mkdir("val_dir")

In [57]:
train_list = list(df_train['id'])
val_list= list(df_val['id'])

In [58]:
for i in range(0, 16):
    os.mkdir(os.path.join('train_dir', str(i)))
for i in range(0, 16):
    os.mkdir(os.path.join('val_dir', str(i)))

In [59]:
for image in train_list:
    fname = str(image)+".tif"
    label = train_labels.loc[image, 'label']
    src = os.path.join('./train/train', fname)
    dst = os.path.join('./train_dir', str(label), fname)
    shutil.copyfile(src, dst)

In [60]:
for image in val_list:
    fname = str(image)+".tif"
    label = train_labels.loc[image, 'label']
    src = os.path.join('./train/train', fname)
    dst = os.path.join('./val_dir', str(label), fname)
    shutil.copyfile(src, dst)

# Data creation ends here you can run the below cells

# ***********************************************************************************

# Loading dataset and Model training starts here.

In [5]:
train_path = 'train_dir'
valid_path = "val_dir"
test_path = "./validation/validation"

num_train_samples = len(df_train)
num_val_samples = len(df_val)

train_batch_size = 24
val_batch_size = 24

train_steps = np.ceil(num_train_samples/train_batch_size)
val_steps = np.ceil(num_val_samples/val_batch_size)

image_size=128

In [6]:
datagen = ImageDataGenerator(rescale=1.0/255)
train_gen = datagen.flow_from_directory(train_path,
                                       target_size=(image_size, image_size),
                                       batch_size=train_batch_size,
                                       class_mode='sparse'
                                       )
val_gen = datagen.flow_from_directory(valid_path,
                                     target_size=(image_size, image_size),
                                     batch_size=val_batch_size,
                                     class_mode='sparse')

Found 14400 images belonging to 16 classes.
Found 1600 images belonging to 16 classes.


In [7]:
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3

model = Sequential()

model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (128, 128, 3)))

model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(16, activation = "softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 122, 122, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 61, 61, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 59, 59, 64)        18496     
                                                        

In [112]:
model.compile(Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
filepath="model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]
history=model.fit_generator(train_gen, steps_per_epoch=train_steps, validation_data=val_gen,
                            validation_steps = val_steps,
                            epochs=20, verbose=1,
                           callbacks=callbacks_list)

Epoch 1/20


C:\Users\shiva\AppData\Local\Temp/ipykernel_30028/618080758.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_gen, steps_per_epoch=train_steps, validation_data=val_gen,


600/600 [==============================] - ETA: 0s - loss: 2.7738 - accuracy: 0.0647
Epoch 1: val_accuracy improved from -inf to 0.06500, saving model to model.h5
600/600 [==============================] - 556s 926ms/step - loss: 2.7738 - accuracy: 0.0647 - val_loss: 2.7723 - val_accuracy: 0.0650 - lr: 0.0010
Epoch 2/20
 65/600 [==>...........................] - ETA: 8:01 - loss: 2.7728 - accuracy: 0.0731

KeyboardInterrupt: 